<a href="https://colab.research.google.com/github/JedRoundy/Predictive_Learning/blob/main/Data_Retrieval_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
###This project allows a user to enter any city that is in the google maps API and receive a comprehensive graph of the best restaurants in the given cities.

In [ ]:
import pandas as pd
import requests
from google.colab import drive
drive.mount('/content/drive')
import statsmodels.formula.api as smf
import plotly.express as px
import json
import sqlite3

Mounted at /content/drive


In [ ]:
#Initialize variables & dataframes
api_key = 'YOUR-GCP-API_KEY-HERE'
types = 'restaurant'




In [ ]:
##Please only use a portion of the following to test my code. If I send too many api requests I'm gonna get hit with a (small-ish) bill.

#Create list of towns to get geo-cords from
towns = [#'Eagle Mountain, UT', 'Saratoga Springs, UT','Riverton, UT', 'Bluffdale', 'Herriman, UT', 'South Jordan, UT'
         'Vineyard, UT',
         #'Springville, UT', 'Mapleton, UT', 'Spanish Fork, UT', 'Payson, UT', 'Salem, UT', 'Nephi, UT',
         #'St. George, UT', 'Cedar City, UT' ,'Santa Clara, UT', 'Ivins, UT',
         'Orem, UT', 'Provo, UT', 'Lindon, UT',
         'Pleasant Grove, UT', 'American Fork, UT', 'Lehi, UT',
         #'Alpine, UT', 'Highland, UT', 'Draper, UT', 'Holladay, UT',
        # 'Sugarhouse, UT', 'Salt Lake City, UT', 'Logan, UT', 'Brigham City, UT', 'Ogden, UT' ,'Roy, UT', 'West Valley, UT',
         #'Murray, UT', 'Mill Creek, UT', 'Woods Cross, UT'
         ]

In [ ]:
# the api endpoint
url = 'https://maps.googleapis.com/maps/api/geocode/json'

# the api parameters
lat_list = []
long_list = []

for town in towns:
  address = town
  # send the api request

  params = {
    'address': address,
    'key': api_key
}

  response = requests.get(url, params=params)

  # parse the response
  data = response.json()

  # extract the latitude and longitude
  lat_list.append(data['results'][0]['geometry']['location']['lat'])
  long_list.append(data['results'][0]['geometry']['location']['lng'])


In [ ]:
place_df = pd.DataFrame()
results = []
photo_url_list = []
x = 0

for town in towns:
  #Get the api website
  url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat_list[x]},{long_list[x]}&radius=2000&types={types}&name=restaurant&key={api_key}"
  #counter for url
  x += 1

  #get the api response
  response = requests.get(url)
  data = response.json()

  # Convert the JSON response into a dataframe
  results = data["results"]
  new_df = pd.json_normalize(results)

  #Convert the photo reference into a photo_url
  for i in range(new_df):
    try:
      photo_reference = results[i]['photos'][0]['photo_reference']
      photo_url = f"https://maps.googleapis.com/maps/api/place/photo?maxwidth=1000&maxheight=1000&photoreference={photo_reference}&key={api_key}"
      photo_url_list.append(photo_url)
    except:
      photo_url_list.append(None)
  #Add the town column
  new_df['town'] = town



  #Append the iterative df to the larger df
  place_df = place_df.append(new_df, ignore_index = True)

# Append the photo url to the dataframe
place_df['photo_url'] = photo_url_list


In [ ]:
#create a list from my place_df to iterate through to grab details of each place.
place_id_list = place_df['place_id'].tolist()

In [ ]:
#initialize variables for my details api request
author_name = []
text = []
author_rating = []
details_df = pd.DataFrame()


for place in place_id_list:
  place_id = place
  url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={api_key}"

  response = requests.get(url)
  data = response.json()

  # Extract the information from the response
  place_details = data["result"]


  #Get the most relevant review for each restaurant
  try:
    author_name.append(place_details['reviews'][0]['author_name'])
  except:
    author_name.append(None)
  try:
    text.append(place_details['reviews'][0]['text'])
  except:
    text.append(None)
  try:
    author_rating.append(place_details['reviews'][0]['rating'])
  except:
    author_rating.append(None)



  new_df = pd.json_normalize(place_details)
  details_df = details_df.append(new_df, ignore_index = True)

details_df['author_name'] = author_name
details_df['author_rating'] = author_rating
details_df['text'] = text

In [ ]:
####
#EDITING AND MERGING DF'S
####


In [ ]:
#Identify wanted columns
details_df = details_df[['place_id','delivery' ,'dine_in','takeout' ,'url', 'website', 'author_name', 'author_rating', 'text']]


In [ ]:
#identify wanted columns
place_df = place_df[['business_status', 'name', 'place_id', 'price_level', 'rating', 'user_ratings_total', 'vicinity', 'town', 'photo_url']]

In [ ]:
#merge columns on place_id
big_df = place_df.merge(details_df, on = 'place_id')

In [ ]:
#remove duplicates
big_df = big_df.loc[big_df['duplicate'] == False]

In [ ]:
#print it
big_df

In [ ]:
big_df.to_csv('/content/drive/My Drive/data.csv', index=False)

In [ ]:
duplicates = big_df.duplicated()

big_df['duplicate'] = duplicates


In [ ]:
px.scatter(data_frame = df_pl_r, y = 'user_ratings_total', x = 'rating', hover_name = 'name', color = 'price_level', range_color= [0,4])